In [ ]:
!pip install transformers torch
!pip install rouge-score -q
!python -m spacy download en_core_web_lg

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 77.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
import numpy as np
import pandas as pd
import spacy
import nltk
import torch
import matplotlib.pyplot as plt
import networkx as nx
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords as nltk_stopwords
from nltk.cluster.util import cosine_distance
from heapq import nlargest
from string import punctuation
from spacy.lang.en import STOP_WORDS
from rouge_score import rouge_scorer, scoring
from sklearn.metrics.pairwise import cosine_similarity
from transformers import BartTokenizer, BartForConditionalGeneration
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab') # Add this line to download the missing resource
nlp = spacy.load('en_core_web_lg') # Load the larger model
stopwords = list(STOP_WORDS)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
df = pd.read_csv("/content/IK_Trump.csv", engine ='python', encoding='latin-1')
df.head(5)

,ArticleID,ArticleTitle,ArticleText,ArticleSource
0,0,"On the Coronavirus, Pakistans Government Is M...","During a televised broadcast on March 22, Paki...",https://foreignpolicy.com/2020/05/11/on-corona...
1,1,Pakistan Plans Another COVID-19 Lockdown. Will...,The government in Pakistan is considering the ...,https://thediplomat.com/2020/05/pakistan-plans...
2,2,COVID-19: Pakistan has a dependency syndrome...,As Pakistan grapples to combat coronavirus wit...,https://gulfnews.com/world/asia/pakistan/covid...
3,3,COVID19 and Pakistan: The Economic Fallout,As Pakistan grapples to combat coronavirus wit...,https://www.orfonline.org/research/covid19-and...
4,4,Donald Trump says hes taking hydroxychloroqui...,President Donald Trump said hes taking antima...,https://www.theverge.com/2020/5/18/21262873/tr...


In [ ]:
if 'ReferenceSummary' not in df.columns:
    df['ReferenceSummary'] = df['ArticleText'].apply(lambda x: x[:1000])  # mock ground truth for testing

In [ ]:
def read_article_text(article_id):
    # Access 'ArticleText' column and split by space
    article = df['ArticleText'][article_id].split(" ")
    sentences = []

    # Treat the entire headline as a single sentence for simplicity
    sentences.append([word for word in article if word]) # Add non-empty words as a single sentence

    return sentences

In [ ]:
#word tokenization
from spacy.lang.en import STOP_WORDS
stopwords = list(STOP_WORDS)

# Define 'doc' using nlp and the first article's text
article_text = df['ArticleText'][0]
doc = nlp(article_text)

tokens = [token.text for token in doc]
print(tokens)
from string import punctuation
punctuation = set(punctuation + '\n')
word_frequencies = {}
for word in doc:
    if word.text.lower() not in stopwords:
        if word.text.lower() not in punctuation:
            if word.text not in word_frequencies.keys():
                word_frequencies[word.text] = 1
            else:
                word_frequencies[word.text] += 1
print(word_frequencies)

['During', 'a', 'televised', 'broadcast', 'on', 'March', '22', ',', 'Pakistani', 'Prime', 'Minister', 'Imran', 'Khan', 'expressed', 'his', 'hesitancy', 'in', 'imposing', 'a', 'nationwide', 'lockdown', 'in', 'response', 'to', 'the', 'coronavirus', 'pandemic', ',', 'explaining', 'that', 'such', 'a', 'move', 'would', 'have', 'devastating', 'economic', 'consequences', 'for', 'the', 'poor', '.', '\x93Twenty', 'five', 'percent', 'of', 'Pakistanis', 'are', 'below', 'the', 'poverty', 'line,\x94', 'Khan', 'said', '.', ' ', '\x93Today', 'if', 'I', 'impose', 'a', 'complete', 'lockdown', ',', 'then', 'my', 'country\x92s', 'rickshaw', 'drivers', ',', 'pushcart', 'vendors', ',', 'taxi', 'drivers', ',', 'small', 'shopkeepers', ',', 'daily', 'wage', 'earners', 'all', 'of', 'them', 'will', 'be', 'shut', 'in', 'their', 'homes', '.', '\x94In', 'Pakistan', ',', 'where', 'around', '30', 'percent', 'of', 'the', 'population', 'lives', 'in', 'grave', 'poverty', ',', 'avoiding', 'catching', 'the', 'coronavirus

In [ ]:
#sentence tokenization
max_frequency = max(word_frequencies.values())
max_frequency
for word in word_frequencies.keys():
    word_frequencies[word] = word_frequencies[word]/max_frequency
print(word_frequencies)
sentence_tokens = [sent for sent in doc.sents]
print(sentence_tokens)

{'televised': 0.04, 'broadcast': 0.04, 'March': 0.08, '22': 0.04, 'Pakistani': 0.08, 'Prime': 0.04, 'Minister': 0.04, 'Imran': 0.04, 'Khan': 0.24, 'expressed': 0.04, 'hesitancy': 0.04, 'imposing': 0.04, 'nationwide': 0.04, 'lockdown': 0.32, 'response': 0.04, 'coronavirus': 0.32, 'pandemic': 0.28, 'explaining': 0.04, 'devastating': 0.04, 'economic': 0.16, 'consequences': 0.04, 'poor': 0.08, '\x93Twenty': 0.04, 'percent': 0.12, 'Pakistanis': 0.08, 'poverty': 0.12, 'line,\x94': 0.04, 'said': 0.08, ' ': 1.0, '\x93Today': 0.04, 'impose': 0.04, 'complete': 0.12, 'country\x92s': 0.16, 'rickshaw': 0.04, 'drivers': 0.08, 'pushcart': 0.04, 'vendors': 0.04, 'taxi': 0.04, 'small': 0.04, 'shopkeepers': 0.04, 'daily': 0.08, 'wage': 0.04, 'earners': 0.04, 'shut': 0.04, 'homes': 0.04, '\x94In': 0.04, 'Pakistan': 0.36, '30': 0.04, 'population': 0.04, 'lives': 0.08, 'grave': 0.04, 'avoiding': 0.04, 'catching': 0.04, 'isn\x92t': 0.04, 'thing': 0.04, 'people\x92s': 0.04, 'minds': 0.04, 'Staving': 0.04, 'h

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from transformers import pipeline
import torch
import networkx as nx # Import networkx

def sentence_vector(sentence):
    doc = nlp(sentence)
    return doc.vector

def sentence_similarity(sent1, sent2):
    return sent1.similarity(sent2)


def build_similarity_matrix(sentences):
    n = len(sentences)
    sim_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            if i != j:
                sim_matrix[i][j] = sentence_similarity(sentences[i], sentences[j])
    return sim_matrix

def textrank_summary(text, top_n=3):
    # Process the entire text with spaCy
    doc = nlp(text)
    # Get spaCy sentence objects
    sentences = list(doc.sents)

    if len(sentences) < top_n:

        return text if sentences else "No sentences to summarize."

    sim_matrix = build_similarity_matrix(sentences)
    sim_graph = nx.from_numpy_array(sim_matrix)
    scores = nx.pagerank(sim_graph)

    # Rank sentences - use original spaCy sentence objects to get text later
    ranked_sentences = sorted(((scores[i], s) for i, s in enumerate(sentences)), reverse=True)
    select_length = int(len(sentences) * top_n) # Calculate based on the actual number of sentences
    summary = ' '.join([s.text for _, s in ranked_sentences[:select_length]]) # Use .text to get string representation
    return summary


# Abstractive Summary

from transformers import BartTokenizer, BartForConditionalGeneration
import torch
import pandas as pd

# Load BART model and tokenizer (Seq2Seq model)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn").to(device)

# Sequence-to-Sequence Abstractive Summarization Function
def abstractive_summary_seq2seq(text, max_length=150, min_length=40):
    try:
        if not text or len(text.strip()) < 30:
            return "Text too short for summarization."

        # Encode the input text (acts as the encoder input)
        inputs = tokenizer(
            text,
            return_tensors="pt",
            max_length=1024,  # BART max input size
            truncation=True
        ).to(device)

        #Generate output tokens (decoder output) using BART
        summary_ids = model.generate(
            input_ids=inputs["input_ids"],       # Encoder input
            attention_mask=inputs["attention_mask"], # Encoder attention
            max_length=max_length,
            min_length=min_length,
            num_beams=4,        # Beam search for quality
            length_penalty=2.0, # Penalize longer sequences
            early_stopping=True # Stop when finished
        )

        # 3️⃣ Decode the output tokens into readable summary
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        return summary

    except Exception as e:
        return f"Abstractive summarization failed due to an error: {e}"


article_id = int(input(f"Enter article index (0 to {len(df)-1}): "))
article_text = df['ArticleText'][article_id]

print(f"\n🔹 Extractive Summary (TextRank):\n{textrank_summary(article_text)}")
print(f"\n🔹 Abstractive Summary (BART):\n{abstractive_summary_seq2seq(article_text)}")



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

Enter article index (0 to 5): 1

🔹 Extractive Summary (TextRank):
With the virus spread now hitting all corners of the country and transmission patterns developing in rural areas, it is unclear if another lockdown is the solution or if it can be implemented at all. However, one can argue that one week of the Eid holidays and the lifting of the lockdown that allowed millions of workers to travel back to their villages from major urban zones may have worsened the problem. On the other hand, in the countrys rural areas, no effort whatsoever is being made to either educate people of the viruss threat or to test, track, and record cases. Smaller cities and adjacent towns across Pakistan, which are not the focus of the public health officials, neither have the capacity nor infrastructure in place to handle COVID-19 cases or record them correctly for the national or international audience. For instance, there is still an ongoing debate about whether the virus spread has taken place in Pak

In [ ]:
#  Evaluate with ROUGE
from rouge_score import rouge_scorer

# Sample reference and generated summaries
reference_summary = "Text summarization shortens a document to its key points."
generated_summary = "Summarization reduces the length of a document."

# Initialize ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Compute scores
scores = scorer.score(reference_summary, generated_summary)

# Print precision, recall, and F1 for each ROUGE metric
for rouge_type, score in scores.items():
    print(f"{rouge_type.upper()}:")
    print(f"  Precision: {score.precision:.4f}")
    print(f"  Recall:    {score.recall:.4f}")
    print(f"  F1 Score:  {score.fmeasure:.4f}")
    print()


ROUGE1:
  Precision: 0.4286
  Recall:    0.3333
  F1 Score:  0.3750

ROUGE2:
  Precision: 0.1667
  Recall:    0.1250
  F1 Score:  0.1429

ROUGEL:
  Precision: 0.4286
  Recall:    0.3333
  F1 Score:  0.3750

